In [2]:
import numpy as np
import pandas as pd
pd.options.display.max_columns=100
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
import sklearn.metrics
from sklearn.preprocessing import StandardScaler as SSc
import torch
from torch import nn, optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

#set width of window to preference
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
data = pd.read_csv("Data-Prepped.csv",index_col=0)
data = data.astype(np.float32)
data.head()

,Bronze,Silver,Gold,Platinum,Diamond,Master,GrandMaster,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,27.0,10.0,3000.0,143.718002,0.003515,0.000220,7.0,0.000110,0.000392,0.004849,32.667702,40.867298,4.7508,28.0,0.001397,6.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,5.0,23.0,10.0,5000.0,129.232193,0.003304,0.000259,4.0,0.000294,0.000432,0.004307,32.919399,42.345402,4.8434,22.0,0.001194,5.0,0.0,0.000208
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0,30.0,10.0,200.0,69.961197,0.001101,0.000336,4.0,0.000294,0.000461,0.002926,44.647499,75.354797,4.0430,22.0,0.000745,6.0,0.0,0.000189
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,19.0,20.0,400.0,107.601601,0.001034,0.000213,1.0,0.000053,0.000543,0.003783,29.220301,53.735199,4.9155,19.0,0.000426,7.0,0.0,0.000384
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,32.0,10.0,500.0,122.890800,0.001136,0.000327,2.0,0.000000,0.001329,0.002368,22.688499,62.081299,9.3740,15.0,0.001174,4.0,0.0,0.000019


In [5]:
X = data.iloc[:,data.columns != 'APM']
Y = data.iloc[:,data.columns == 'APM']
#transform input data (normalize scaling)
ssc = SSc()
Xft = ssc.fit_transform(X)
X = pd.DataFrame(Xft)
print("Xtr(Xtrain),Xtst(Xtest),Ytr(Ytrain),Ytst(Ytest) shapes: ")
Xtr,Xtst,Ytr,Ytst = train_test_split(X,Y.values.ravel(),test_size=0.2,random_state=2020)
print(Xtr.shape,Xtst.shape,Ytr.shape,Ytst.shape)
Ytr = pd.DataFrame(Ytr)

Xtr(Xtrain),Xtst(Xtest),Ytr(Ytrain),Ytst(Ytest) shapes: 
(2670, 25) (668, 25) (2670,) (668,)


In [6]:
x_tr = torch.tensor(Xtr.values.astype(np.float64))
y_tr = torch.tensor(Ytr.values.astype(np.float32))
x_tst = torch.tensor(Xtst.values.astype(np.float64))
y_tst = torch.tensor(Ytst.astype(np.float32))

btch_sz = 20

tr_dset = TensorDataset(x_tr,y_tr)
tr_dload = DataLoader(dataset=tr_dset, batch_size=btch_sz, shuffle=True)

tst_dset = TensorDataset(x_tst,y_tst)
tst_dload = DataLoader(dataset=tst_dset, batch_size=btch_sz)

In [7]:
'''
class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * torch.sigmoid(i)
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = torch.sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))
    
class swish(nn.Module):
    def forward(self, input_tensor):
        return Swish.apply(input_tensor)
'''


class resultNet(nn.Module):
    def __init__(self, X_sz, Y_sz, a=0, b=0, c=0, d=0):
        super(resultNet, self).__init__()
        
        self.inputSize = len(X.columns)
        self.outputSize = len(Y.columns)
        self.hidden0Size = a
        self.hidden1Size = b
        self.hidden2Size = c
        self.hidden3Size = d
        self.activation = F.selu
        self.outactivation = F.selu
        self.outsquish = torch.sigmoid
        
        #Connect network
        self.dpth = 0
        if (self.hidden0Size != 0):
            self.c1 = nn.Linear(self.inputSize,self.hidden0Size)
            self.dpth += 1
            print("adding layer 1")
            if (self.hidden1Size != 0):
                self.c2 = nn.Linear(self.hidden0Size,self.hidden1Size)
                self.dpth += 1
                print("adding layer 2")
                if (self.hidden2Size != 0):
                    self.c3 = nn.Linear(self.hidden1Size,self.hidden2Size)
                    self.dpth += 1
                    print("adding layer 3")
                    if (self.hidden3Size != 0):
                        self.c4 = nn.Linear(self.hidden2Size,self.hidden3Size)
                        self.dpth += 1
                        print("adding layer 4")
                        self.c5 = nn.Linear(self.hidden3Size,self.outputSize)
                    else:
                        self.c4 = nn.Linear(self.hidden2Size,self.outputSize)
                else:
                    self.c3 = nn.Linear(self.hidden1Size,self.outputSize)
            else:
                self.c2 = nn.Linear(self.hidden0Size,self.outputSize)
        else:
            self.c1 = nn.Linear(self.inputSize,self.outputSize)
        
    def forward(self, x):
        
        if (self.dpth == 0):
            out = self.outsquish(self.outactivation(self.c1(x)))
            #print("fwd dpth 0")
        elif (self.dpth == 1):
            x = self.activation(self.c1(x))
            out = self.outsquish(self.outactivation(self.c2(x)))
            #print("fwd dpth 1")
        elif (self.dpth == 2):
            x = self.activation(self.c1(x))
            x = self.activation(self.c2(x))
            out = self.outsquish(self.outactivation(self.c3(x)))
            #print("fwd dpth 2")
        elif (self.dpth == 3):
            x = self.activation(self.c1(x))
            x = self.activation(self.c2(x))
            x = self.activation(self.c3(x))
            out = self.outsquish(self.outactivation(self.c4(x)))
            #print("fwd dpth 3")
        elif (self.dpth == 4):
            x = self.activation(self.c1(x))
            x = self.activation(self.c2(x))
            x = self.activation(self.c3(x))
            x = self.activation(self.c4(x))
            out = self.outsquish(self.outactivation(self.c5(x)))
            #print("fwd dpth 4")
        return out
    
h_size = 12
testNet = resultNet(len(Xtr.columns),len(Ytr.columns),25)
for p in testNet.parameters():
    print(p)

adding layer 1
Parameter containing:
tensor([[-4.9726e-02,  7.2450e-02, -1.2798e-02,  6.9392e-02,  4.2379e-03,
          1.5457e-01, -8.1887e-02,  4.7553e-02, -1.8418e-01, -6.9317e-02,
          4.4955e-02,  3.7756e-02, -1.7748e-01,  1.6583e-01,  6.9771e-02,
         -1.5166e-01, -5.2824e-02, -1.4619e-01, -1.2801e-01, -6.1047e-02,
         -1.0535e-01, -1.3497e-01, -4.7729e-02, -4.6701e-03, -8.0913e-02],
        [-1.9027e-02, -1.8105e-01,  1.6733e-01, -8.6210e-02,  1.3645e-01,
         -5.0487e-02, -1.0134e-02, -1.2486e-01, -5.8060e-02,  3.8344e-02,
          1.4308e-03,  1.4626e-01,  6.9895e-02, -1.5107e-01, -1.4297e-01,
          2.5415e-03, -1.0102e-01,  1.9226e-01,  8.1118e-02, -1.3433e-01,
         -6.0465e-02,  2.0874e-02,  8.3659e-02, -1.3880e-01, -2.6513e-02],
        [-6.5117e-02, -5.8546e-02,  9.3555e-02, -6.2392e-02,  4.5130e-02,
         -1.6837e-01, -1.7615e-01,  1.4988e-01,  1.7800e-01, -1.5659e-01,
         -8.7009e-02, -1.3793e-01, -9.8461e-02, -6.4829e-02,  1.7095e-01,

In [8]:
def test(model, lss_fn, tst_dload):
    scores = []
    with torch.no_grad():
        model.eval()
        for (x_btch, y_btch) in tst_dload:
            out_btch = model(x_btch.float())
            lss = lss_fn(out_btch.float()[:,0], y_btch.long())
            scores.append(lss.item())
        model.train()
    return np.array(scores).mean()

test(testNet, nn.MSELoss(), tst_dload)

14623.01309742647

In [9]:
rnet = resultNet(Xtr,Ytr)
print(rnet)

learn_rate = 1
inertia = .8

criterion = nn.MSELoss()
optimizer = optim.SGD(rnet.parameters(), lr=learn_rate, momentum = inertia)


gpu_rdy = torch.cuda.is_available()
if gpu_rdy:

    print("Using GPU")
else:
    print("Using CPU")

resultNet(
  (c1): Linear(in_features=25, out_features=1, bias=True)
)
Using GPU


In [10]:
device = torch.device("cuda" if gpu_rdy else "cpu")

OGscr = test(rnet, criterion, tst_dload)

n_epochs = 201
idx = 0

tr_shp = Xtr.shape[0]

X_tr = torch.from_numpy(Xtr.values)
X_tr.to(device)

t_epochs = 0

In [11]:
#you can keep iterating this block to continue training the network

if gpu_rdy:
    print("On_GPU")
print("\nDisplayed score is MSE on 289 test data points while model is trained on 1155 training data points\n")
rnet.train() #just in case

print("\nUntrained score:            {}\n".format(OGscr))

lr_ = lambda epoch: (0.95 ** epoch)/10
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_)
for epoch in range(n_epochs):
    
    '''
    if idx + btch_sz >= tr_shp:
        idx = 0
    else:
        idx += btch_sz
    
    x_tr = Variable(x_tr[idx:(idx+btch_sz)].clone())
    '''
    
    
    for i, (x_btch, y_btch) in enumerate(tr_dload):
        if gpu_rdy:
            rnet.to(device)
            x_btch = x_btch.cuda()
            y_btch = y_btch.cuda()

        optimizer.zero_grad()

        out_btch = rnet(x_btch.float())
        out_lss = criterion(out_btch, y_btch)
        
        out_lss.backward()
        optimizer.step()
    t_epochs += 1
    rnet.to('cpu')
    scr = test(rnet, criterion, tst_dload)
    if (epoch %5) == 0:
        print("epoch {:06d} test data score: {}".format(t_epochs,scr))
    t_epochs += 1

On_GPU

Displayed score is MSE on 289 test data points while model is trained on 1155 training data points


Untrained score:            14624.968778722427

epoch 000001 test data score: 14514.310403262867
epoch 000011 test data score: 14514.310345818014
epoch 000021 test data score: 14514.310288373163
epoch 000031 test data score: 14514.310288373163
epoch 000041 test data score: 14514.310288373163
epoch 000051 test data score: 14514.310259650736
epoch 000061 test data score: 14514.310259650736
epoch 000071 test data score: 14514.310259650736
epoch 000081 test data score: 14514.310259650736
epoch 000091 test data score: 14514.310259650736
epoch 000101 test data score: 14514.310259650736
epoch 000111 test data score: 14514.310259650736
epoch 000121 test data score: 14514.31023092831
epoch 000131 test data score: 14514.31023092831
epoch 000141 test data score: 14514.31023092831
epoch 000151 test data score: 14514.31023092831
epoch 000161 test data score: 14514.31023092831
epoch 000171 te

In [12]:
tr_scr = test(rnet, criterion, tr_dload)
print("Score on training data for comparison: {}".format(tr_scr))


Score on training data for comparison: 15276.030535797574


C:\Users\Triplea657\anaconda3\envs\MSCS335\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([20, 1])) that is different to the input size (torch.Size([20])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Triplea657\anaconda3\envs\MSCS335\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


##### I wasn't really able to find anything interesting with neural networks on this data. Perhaps if I had planned out my schedule better and had devoted more time to just letting my computer sit and crunch while I worked on other things, I would've been able to test more.